### Test ADWH connection

In [1]:
import cx_Oracle as cxo

from ads.database import connection 
from ads.database.connection import Connector
from ads.dataset.factory import DatasetFactory
from ads.vault.vault import Vault
from sqlalchemy import create_engine
from urllib.request import urlopen
import os
import warnings
import logging

warnings.filterwarnings("ignore", category=DeprecationWarning) 
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)

In [2]:
print(cxo.__version__)

8.3.0


In [3]:
#credential = {'database_name': 'dbtst1_medium', 
#              'username': 'DEMO1', 
#              'password': 'Pippo12345::',
#              'database_type':'oracle'}

In [4]:
# memorizzo in un repository locale
# in questo modo nel NB non c'è la pwd
# che ho memorizzato prima nel repo (vedi istruzione commentata)
database_name = "dbtst1_medium"

# connection.update_repository(key=database_name, value=credential) 
connection.get_repository(key=database_name)

{'database_type': 'oracle',
 'database_name': 'dbtst1_medium',
 'username': 'DEMO1',
 'password': 'Pippo12345::'}

In [5]:
# import the wallet
wallet_path = os.path.join(os.path.expanduser("~/data-science-bp"), f"Wallet_{database_name}.zip")

if database_name != "<database_name>":
    connection.import_wallet(wallet_path=wallet_path, key=database_name)

In [6]:
if "database_name" in globals() and database_name != '<database_name>':
    connector = Connector(key=database_name)
else:
    print("Skipping as it appears that you do not have database_name configured.")

In [7]:
if "connector" in globals():
    print(connector.config)
else:
    print("Skipping as it appears that you do not have connector configured.")

{'database_type': 'oracle', 'database_name': 'dbtst1_medium', 'username': 'DEMO1', 'password': 'Pippo12345::'}


In [8]:
query = """
    SELECT * from CUSTOMERS
    """

In [9]:
%%time
if "connector" in globals():
    print(f'uri is: {connector.uri}')
    
    customers = DatasetFactory.open(connector.uri, format="sql", 
                                     table=query)
else:
    print("Skipping as it appears that you do not have connector configured.")

uri is: oracle+cx_oracle://DEMO1:Pippo12345::@dbtst1_medium


loop1:   0%|          | 0/4 [00:00<?, ?it/s]


0 rows [00:00, ? rows/s]
12007 rows [00:00, 24132.53 rows/s]
24014 rows [00:00, 35401.45 rows/s]
36021 rows [00:00, 39528.61 rows/s]
48028 rows [00:01, 39707.88 rows/s]
55500 rows [00:01, 34881.22 rows/s]


CPU times: user 2.91 s, sys: 206 ms, total: 3.12 s
Wall time: 3.42 s


In [10]:
customers.head()

,cust_id,cust_first_name,cust_last_name,cust_gender,cust_year_of_birth,cust_marital_status,cust_street_address,cust_postal_code,cust_city,cust_city_id,...,cust_main_phone_number,cust_income_level,cust_credit_limit,cust_email,cust_total,cust_total_id,cust_src_id,cust_eff_from,cust_eff_to,cust_valid
0,49671,Abigail,Ruddy,M,1976,married,27 North Sagadahoc Boulevard,60332,Ede,51442,...,519-236-6123,"G: 130,000 - 149,999",1500,Ruddy@company.example.com,Customer total,52772,None,1998-01-01,None,I
1,3228,Abigail,Ruddy,M,1964,None,37 West Geneva Street,55406,Hoofddorp,51669,...,327-194-5008,"G: 130,000 - 149,999",7000,Ruddy@company.example.com,Customer total,52772,None,1998-01-01,None,A
2,6783,Abigail,Ruddy,M,1942,single,47 Toa Alta Road,34077,Schimmert,52297,...,288-613-9676,"G: 130,000 - 149,999",11000,Ruddy@company.example.com,Customer total,52772,None,1998-01-01,None,I
3,10338,Abigail,Ruddy,M,1977,married,47 South Kanabec Road,72996,Scheveningen,52296,...,222-269-1259,"G: 130,000 - 149,999",1500,Ruddy@company.example.com,Customer total,52772,None,1998-01-01,None,I
4,13894,Abigail,Ruddy,M,1949,None,57 North 3rd Drive,67644,Joinville,51698,...,675-133-2226,"G: 130,000 - 149,999",9000,Ruddy@company.example.com,Customer total,52772,None,1998-01-01,None,I


In [11]:
customers.shape

(55500, 23)